# Selenium web scraping
- 1: Get list of all labels
- 2: Get list of all endpoints for all labels
- 3: Do all requests for first image.
- 4: Sort this list
- 5: take the highest x (20?) requests and send all other images to that endpoint
- 6: sort that list, which is now the highest?
optional: automatic send the answer?

- 7: if wrong, ping endpoint each minute untill other value (so rest) and do the same again

# 1: Get list of all labels

In [45]:
# Get all labels to print
import json

# return the wordnetsysnet
def get_all_labels_name():
    f = open(r"C:\Users\niels\Downloads\Reverse Captcha\labels.json")
  
    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    labels_to_get = []
    for i in range(0, 1000): # because 1000 records
        #print(data[i]['label'])
        labels_to_get.append(data[i]['label'])
    f.close()
    #print(labels_to_get)
    return labels_to_get

item_labels = get_all_labels_name()

# return the wordnetsysnet
def get_all_labels_wordnetSynset():
    f = open(r"C:\Users\niels\Downloads\Reverse Captcha\labels.json")
  
    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    labels_to_get = []
    for i in range(0, 1000): # because 1000 records
        #print(data[i]['label'])
        labels_to_get.append(data[i]['wordnetSynset'])
    f.close()
    #print(labels_to_get)
    return labels_to_get

wordnetSynset_labels = get_all_labels_wordnetSynset()

# 2: Get list of all endpoints for all labels

In [3]:
# create list of all image names per category (will be faster then calling os for each request)
import os

class ImageLocationItem:
    label: int
    location1: str
    location2: str
    location3: str
    location4: str
    location5: str
    def __init__(self):
        self.location1 = ""
        self.location2 = ""
        self.location4 = ""
        self.location3 = ""
        self.location5 = ""
    def PrintItem(self):
        print("for " + str(self.label))
        print("1: " + self.location1)
        print("2: " + self.location2)
        print("3: " + self.location3)
        print("4: " + self.location4)
        #print("5: " + self.location5) # most images only have max of 4 images available in directory
    def GetLocationList(self):
        locations = []
        if self.location1 != None:
            locations.append(self.location1)
        if self.location2 != None:
            locations.append(self.location2)
        if self.location3 != None:
            locations.append(self.location3)
        if self.location4 != None:
            locations.append(self.location4)
        return locations

item_labels = get_all_labels_name()
image_location_list = []
for x in range(0, 1000):
    item_name = item_labels[x]
    
    obj = ImageLocationItem()
    obj.label = item_name
    
    directory_in_str = "C:\ReversedCaptcha\hd-label-images\\" + item_name
    directory = os.fsencode(directory_in_str)
    
    currentIndex = 1
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
         
        if currentIndex == 1:
            obj.location1 = directory_in_str + "\\" + filename # + directory? check when first time debugging
        if currentIndex == 2:
            obj.location2 = directory_in_str + "\\" + filename # + directory? check when first time debugging
        if currentIndex == 3:
            obj.location3 = directory_in_str + "\\" + filename # + directory? check when first time debugging
        if currentIndex == 4:
            obj.location4 = directory_in_str + "\\" + filename # + directory? check when first time debugging
        if currentIndex == 5:
            obj.location5 = directory_in_str + "\\" + filename # + directory? check when first time debugging
        
        if currentIndex == 6:
            break
        
        currentIndex += 1
    image_location_list.append(obj)

# Testing
#image_location_list[400].PrintItem()
#x = 1000
#for i in range(0, x): # if this does not fail, all images have at least 4 images 
    #image_location_list[i].PrintItem()

# 3: Do all requests for first image.

In [40]:
# image_location_list contains all image location
# item_labels contains all labels

import requests
import concurrent
from concurrent.futures import ThreadPoolExecutor

loops = range(0, 1000)

failed_labels = []
failed_indexes = []
dirs_to_remove = []

real_url = 'https://europe-west1-coding-challenge-platform.cloudfunctions.net/ctf-reverse-captcha-5736741495373824'

def get_label_info(index):
    print("getting " + str(index))
    # Prepare request to send for index
    label = item_labels[index]
    #image_path = "C:\ReversedCaptcha\Images\itemname\itemname1.jpg".replace("itemname", label) # non HD images
    image_path = image_location_list[index].location1 # HD images
    
    # Open image to send along
    with open(image_path, "rb") as file:
        files = {'file': file}
        
        # Send the request
        response = requests.post(real_url, headers={"Authorization":"{5736741495373824}"}, files=files)
        
        #print(json_response)

        json_response = None
        if response.status_code == 200:
            json_response = response.json()
            #print(json_response["result"] + " " + label + " Successfully retrieved label " + label) BIG_INFO
        elif response.status_code == 422:
            dirs_to_remove.append(image_path) # image is not sendable
        else:
            failed_labels.append(label)
            failed_indexes.append(index)
            #print("Something went wrong for label " + label + " with status code " + str(response.status_code)) BIG_INFO
        
        # Do something with the response
        json_response['label'] = label # add the label to the response data so we can look it up later
        json_response['index'] = index
        
    return json_response

def get_result_for_each_first_image():
    print("Getting results...")
    failed_labels = [] # global info lists to keep track off
    failed_indexes = []
    dirs_to_remove = []
    
    result_list = []
    test_url = 'http://localhost:3000/users'
    real_url = 'https://europe-west1-coding-challenge-platform.cloudfunctions.net/ctf-reverse-captcha-5736741495373824'
    threads = 23
    
    with ThreadPoolExecutor(max_workers=threads) as executor:
        future_to_url = {executor.submit(get_label_info, char) for char in loops}

        for future in concurrent.futures.as_completed(future_to_url):
            try:
                data = future.result()
                #print(data)
                result_list.append(data)
                #print(data['status'])
                #print(data['result'])
            except Exception as e:
                #print("Something went wrong " + str(e))
                pass
    print("Got all single results. Total of " + str(len(result_list)) + " records.")
    return result_list


            
def remove_image_from_dir(directory):
    #print("deleting " + directory)
    os.remove(directory)

In [41]:
play_data = get_result_for_each_first_image()


Getting results...
getting 0
getting 1
getting 2
getting 3
getting 4
getting 5
getting 6
getting 7
getting 8
getting 9
getting 10
getting 11
getting 12
getting 13
getting 14
getting 15
getting 16
getting 17
getting 18
getting 19
getting 20
getting 21
getting 22
getting 23
getting 24
getting 25
getting 26
getting 27
getting 28
getting 29
getting 30
getting 31
getting 32
getting 33
getting 34
getting 35
getting 36
getting 37
getting 38
getting 39
getting 40
getting 41
getting 42
getting 43
getting 44
getting 45
getting 46
getting 47
getting 48
getting 49
getting 50
getting 51
getting 52
getting 53
getting 54
getting 55
getting 56
getting 57
getting 58
getting 59
getting 60
getting 61
getting 62
getting 63
getting 64
getting 65
getting 66
getting 67
getting 68
getting 69
getting 70
getting 71
getting 72
getting 73
getting 74
getting 75
getting 76
getting 77
getting 78
getting 79
getting 80
getting 81
getting 82
getting 83
getting 84
getting 85
getting 86
getting 87
getting 88
getting 89
g

getting 692
getting 693
getting 694
getting 695
getting 696
getting 697
getting 698
getting 699
getting 700
getting 701
getting 702
getting 703
getting 704
getting 705
getting 706
getting 707
getting 708
getting 709
getting 710
getting 711
getting 712
getting 713
getting 714
getting 715
getting 716
getting 717
getting 718
getting 719
getting 720
getting 721
getting 722
getting 723
getting 724
getting 725
getting 726
getting 727
getting 728
getting 729
getting 730
getting 731
getting 732
getting 733
getting 734
getting 735
getting 736
getting 737
getting 738
getting 739
getting 740
getting 741
getting 742
getting 743
getting 744
getting 745
getting 746
getting 747
getting 748
getting 749
getting 750
getting 751
getting 752
getting 753
getting 754
getting 755
getting 756
getting 757
getting 758
getting 759
getting 760
getting 761
getting 762
getting 763
getting 764
getting 765
getting 766
getting 767
getting 768
getting 769
getting 770
getting 771
getting 772
getting 773
getting 774
gett

In [43]:
class ScientificSortable:
    result: float
    label: str
        
def list_to_scientific_sortables(input_list):
    sortables = []
    
    for rec in input_list:
        to_add = ScientificSortable()
        to_add.result = float(rec['result'])
        to_add.label = rec['label']
        sortables.append(to_add)
    
    return sortables

def print_sortable_list(sortable_list):
    for x in sortable_list:
        print(str(x.result) + " for label " + x.label)
        
def sort_sortable_list_by_scientific_float(sortable_list):
    sortable_list.sort(key=lambda x: x.result, reverse=True)
    print("From First Try")
    print_sortable_list(sortable_list)
    return sortable_list

#sor = list_to_scientific_sortables(play_data)
#sorted_list = sort_sortable_list_by_scientific_float(sor)

From First Try
0.12952588 for label notebook
0.09685103 for label hard_disc
0.08094742 for label steel_drum
0.034960147 for label tape_player
0.0280168 for label toaster
0.016272143 for label scoreboard
0.014003337 for label cassette_player
0.007820777 for label switch
0.0075252666 for label puck
0.006987676 for label modem
0.006866199 for label face_powder
0.0065658195 for label tray
0.0048484043 for label oil_filter
0.0044894507 for label CD_player
0.0028965573 for label gas_pump
0.0025404117 for label crate
0.002155995 for label pillow
0.0021255708 for label menu
0.0020223993 for label vault
0.0018981 for label Petri_dish
0.0018857073 for label spotlight
0.0018637747 for label basset
0.0018291177 for label Lhasa
0.0018160125 for label loupe
0.0016550829 for label knee_pad
0.0016187254 for label desktop_computer
0.0015890929 for label eft
0.0014910048 for label iron
0.0014246696 for label whistle
0.0013727862 for label purse
0.0013051602 for label laptop
0.0012645138 for label medici

In [38]:
#print_sortable_list(sorted_list)

0.74406004 for label Petri_dish
0.018768629 for label washer
0.0069385087 for label steel_drum
0.0062033474 for label plate
0.0047419104 for label hotdog
0.002410315 for label manhole_cover
0.0024046567 for label thatch
0.0022477326 for label file
0.0014948891 for label torch
0.0014669442 for label seat_belt
0.0014416747 for label conch
0.0011933213 for label planetarium
0.0011235212 for label saltshaker
0.0010972269 for label wreck
0.0010314215 for label laptop
0.0010309282 for label bolete
0.001026243 for label toilet_seat
0.0010125282 for label buckeye
0.001005509 for label potter's_wheel
0.0009844808 for label European_gallinule
0.0009699143 for label lens_cap
0.0009377659 for label paintbrush
0.0009186295 for label sleeping_bag
0.00089889864 for label lampshade
0.0008885155 for label confectionery
0.00087510527 for label bucket
0.000873189 for label wine_bottle
0.00084987935 for label brass
0.00081353064 for label pier
0.00079537573 for label burrito
0.0007743942 for label mortar


# 4: Sort this list with first requests

In [98]:
def get_n_highest_of_list(result_list, n):
    print("sorting list of results with " +  str(len(result_list)) + " elements")

    result_list.sort(key=lambda x: x["result"])
    result_list.reverse()
    
    # print top x from the bottom (sort does sort )
    for i in range(0, 100):
        print(result_list[i])
        
    return result_list[0:n]

def sort_advanced_list(advanced_list):
    advanced_list.sort(key=lambda x: x.average_top_three)
    advanced_list.reverse()
    return advanced_list

In [99]:
# Remove false directories
def remove_false_dirs():
    print("n dirs to remove: " + str(len(dirs_to_remove)))
    for d in dirs_to_remove:
        remove_image_from_dir(d)

# 5: take the highest x (20?) requests and send all other images to that endpoint

In [61]:
class AdvancedLabelData:
    label_index: int
    label_name: str
    max_value: float
    min_value: float
    average_total: float
    average_top_three: float
    total_dirs: int
    def LogThis(self):
        print("label_name: " + self.label_name)
        print("label_index: " + str(self.label_index))
        print("max_value: " + str(self.max_value))
        print("min_value: " + str(self.min_value))
        print("average_total: " + str(self.average_total))
        print("average_top_three: " + str(self.average_top_three))
        print("total_dirs: " + str(self.total_dirs))
        print("--------------------")

def get_advanced_label_data(label_name, index):
    all_results = get_all_label_results(label_name, index)
    #print(all_results)
    all_results_as_floats = str_list_to_float(all_results)
    all_results_as_floats.sort(reverse=True)
    
    advanced_data = AdvancedLabelData()
    
    advanced_data.label_index = index
    advanced_data.label_name = label_name
    advanced_data.total_dirs = len(all_results)
    
    advanced_data.max_value = all_results_as_floats[0]
    advanced_data.min_value = all_results_as_floats[len(all_results_as_floats)-1]
    
    # if 3+ records are in response set
    if len(all_results) >= 3:
        advanced_data.average_total = calc_average_total(all_results_as_floats)
        advanced_data.average_top_three = calc_average_total(all_results_as_floats[0:3])
    
    # if 2 records
    if len(all_results) == 2:
        advanced_data.average_total = calc_average_total(all_results_as_floats)
        advanced_data.average_top_three = calc_average_total(all_results_as_floats[0:2])
    
    # if only 1 is in response set
    if len(all_results) == 1:
        advanced_data.average_total = all_results_as_floats[0]
        advanced_data.average_top_three = None
    
    return advanced_data

def get_advanced_label_data_by_index(index):
    label_name = item_labels[index]
    
    all_results = get_all_label_results(label_name, index)
    
    all_results_as_floats = str_list_to_float(all_results)
    all_results_as_floats.sort(reverse=True)
    advanced_data = AdvancedLabelData()
    
    advanced_data.label_index = index
    advanced_data.label_name = label_name
    advanced_data.total_dirs = len(all_results)
    
    advanced_data.max_value = all_results_as_floats[0]
    advanced_data.min_value = all_results_as_floats[len(all_results_as_floats)-1]
    
    # if 3+ records are in response set
    if len(all_results) >= 3:
        advanced_data.average_total = calc_average_total(all_results_as_floats)
        advanced_data.average_top_three = calc_average_total(all_results_as_floats[0:3])
   
    # if 2 records
    if len(all_results) == 2:
        advanced_data.average_total = calc_average_total(all_results_as_floats)
        advanced_data.average_top_three = calc_average_total(all_results_as_floats[0:2])
    
    # if only 1 is in response set
    if len(all_results) == 1:
        advanced_data.average_total = all_results_as_floats[0]
        advanced_data.average_top_three = None
    
    return advanced_data

def get_all_label_results(label_name, index):
    label_image_dirs = image_location_list[index].GetLocationList()
    all_label_values = []
    for image_dir in label_image_dirs:
        try:
            resp = get_info_by_image_path(image_dir)
            all_label_values.append(resp["result"][0:4])
        except Exception as e:
            print(":-( " + str(e))
    return all_label_values
    
    
def get_info_by_image_path(image_path):
    # Open image to send along
    with open(image_path, "rb") as file:
        files = {'file': file}
        
        # Send the request
        response = requests.post(real_url, headers={"Authorization":"{5736741495373824}"}, files=files)
        json_response = response.json()
        #print(json_response)

        if response.status_code == 422:
            dirs_to_remove.append(image_path) # image is not sendable   
    return json_response

def str_list_to_float(str_list):
    n_list = []
    
    for s in str_list:
        try:
            n_list.append(float(s))
        except:
            #print("not a float (prob 'Could not parse... error because image cant send correctly'")
            pass
    return n_list

def calc_average_total(number_list):
    total = 0
    
    for number in number_list:
        total += number

    return total / len(number_list)

# play_data = get_first_sorted_tries() play_data can be used for testing
def get_advanced_new_data():
    first_run = get_result_for_each_first_image()
    sc_sorted = list_to_scientific_sortables(first_run)
    sorted_list = sort_sortable_list_by_scientific_float(sc_sorted)
    return sorted_list

def get_top_n_labels(first_try_sorted_items, n):
    return_list = []
    
    for item in first_try_sorted_items[:n]:
        return_list.append(item.label)

    print("First from top n labels is " + return_list[0])
    return return_list

def get_advanced_new_data_2():
    sorted_tries = get_first_sorted_tries()
    top_labels = get_top_n_labels(sorted_tries, 4)
    
    advanced_data = []
    
    for top_label in top_labels:
        print("tt " + str())
        index = item_labels.index(top_label)
        get_advanced_label_data(top_label, index)
    return advanced_data

datas = get_advanced_new_data() # THIS CAPTURED THE FLAG

Getting results...
getting 0
getting 1
getting 2
getting 3
getting 4
getting 5
getting 6
getting 7
getting 8
getting 9
getting 10
getting 11
getting 12
getting 13
getting 14
getting 15
getting 16
getting 17
getting 18
getting 19
getting 20
getting 21
getting 22
getting 23
getting 24
getting 25
getting 26
getting 27
getting 28
getting 29
getting 30
getting 31getting 32

getting 33
getting 34
getting 35
getting 36
getting 37
getting 38
getting 39
getting 40
getting 41
getting 42
getting 43
getting 44
getting 45
getting 46
getting 47
getting 48
getting 49
getting 50
getting 51
getting 52
getting 53
getting 54
getting 55
getting 56
getting 57
getting 58
getting 59
getting 60
getting 61
getting 62
getting 63
getting 64
getting 65
getting 66
getting 67
getting 68
getting 69
getting 70
getting 71
getting 72
getting 73
getting 74
getting 75
getting 76
getting 77
getting 78
getting 79
getting 80
getting 81
getting 82
getting 83
getting 84
getting 85
getting 86
getting 87
getting 88
getting 89
g

getting 692
getting 693
getting 694
getting 695
getting 696
getting 697
getting 698
getting 699
getting 700
getting 701
getting 702
getting 703
getting 704
getting 705
getting 706
getting 707
getting 708
getting 709
getting 710
getting 711
getting 712
getting 713
getting 714
getting 715
getting 716
getting 717
getting 718
getting 719
getting 720
getting 721
getting 722
getting 723
getting 724
getting 725
getting 726
getting 727
getting 728
getting 729
getting 730
getting 731
getting 732
getting 733
getting 734
getting 735
getting 736
getting 737
getting 738
getting 739
getting 740
getting 741
getting 742
getting 743
getting 744
getting 745
getting 746
getting 747
getting 748
getting 749
getting 750
getting 751
getting 752
getting 753
getting 754
getting 755
getting 756
getting 757
getting 758getting 759

getting 760
getting 761
getting 762
getting 763
getting 764
getting 765
getting 766
getting 767
getting 768
getting 769
getting 770
getting 771
getting 772
getting 773
getting 774
gett

In [54]:
print(play_data[0].label)

notebook


In [60]:
#adv_data = get_advanced_new_data()

print(adv_data)

[]


In [101]:
def get_indexes_from_list(the_list):
    index_list = []
    
    for rec in the_list:
        index_list.append(rec["index"])
    
    return index_list

def get_advanced_top_n_list(top_n, all_results):
    first_batch = get_result_for_each_first_image()
    list_top = get_n_highest_of_list(all_results, top_n)

    for record in list_top:
        label_name = record['label']
        index = item_labels.index(label_name)
        a = get_advanced_label_data(label_name, index)
        a.LogThis()
        
def get_adnvanced_data_by_index_list(index_list):
    advanced_result_list = []
    real_url = 'https://europe-west1-coding-challenge-platform.cloudfunctions.net/ctf-reverse-captcha-5736741495373824'
    threads = 23
    
    with ThreadPoolExecutor(max_workers=threads) as executor:
        future_to_url = {executor.submit(get_advanced_label_data_by_index, char) for char in index_list}

        for future in concurrent.futures.as_completed(future_to_url):
            try:
                data = future.result()
                #print(data)
                advanced_result_list.append(data)
                #print(data['status'])
                #print(data['result'])
            except Exception as e:
                print("Something went wrong " + str(e))    
    return advanced_result_list

# 6 send request and check if okay

In [102]:
def send_guess(index):
    label_wordnetsyns = wordnetSynset_labels[index]
    label_name = item_labels[index]
    print("sending guess for label " + label_name + " with synsnet " + label_wordnetsyns)
    return guess_answer(label_name, label_wordnetsyns)
                
def guess_answer(label, wordnet):
    url = "https://europe-west1-coding-challenge-platform.cloudfunctions.net/ctf-reverse-captcha-5736741495373824/answer"
    r = requests.post(url, headers={"Authorization":"{5736741495373824}"}, json={"label": label, "wordnetSynset": wordnet})

    if r.status_code == 500:
        print("Guess was wrong. Status code: " + str(r.status_code) + ". With message: " + str(r.content))
    else:
        print("YOOO, BIG ")
        print(r.status_code)
        print(r.content)
    return r.status_code != 500
    
def calc_labels_and_send_request(print_results):
    first_batch = get_result_for_each_first_image()
    highest_from_list = get_n_highest_of_list(first_batch, 50)
    index_list = get_indexes_from_list(highest_from_list)
    adv_results = get_adnvanced_data_by_index_list(index_list)
    sorted_adv_results = sort_advanced_list(adv_results)

    sorted_adv_results[0].LogThis()
    
    if print_results:
        for res in sorted_adv_results:
            res.LogThis()
            
    answer_correct = send_guess(sorted_adv_results[0].label_index)
    print("answer correct: " + str(answer_correct))
    return answer_correct

# 7: if wrong, ping endpoint each minute untill other value (so rest) and do the same again

In [106]:
import requests
import time
from datetime import datetime

def send_static_request_with_same_image_always():
    # Prepare request to send for index
    label = "n09193705"
    image_path = "C:\ReversedCaptcha\hd-label-images\\accordion\\accordion1.jpg"
    real_url = 'https://europe-west1-coding-challenge-platform.cloudfunctions.net/ctf-reverse-captcha-5736741495373824'
    
    # Open image to send along
    with open(image_path, "rb") as file:
        files = {'file': file}
        
        # Send the request
        response = requests.post(real_url, headers={"Authorization":"{5736741495373824}"}, files=files)
        print("Static ping " + response.json()['result'])
        
        # Do something with the response
        json_response = response.json()
        json_response['label'] = label # add the label to the response data so we can look it up later
    return json_response

def send_request_and_get_response():
    r = send_static_request_with_same_image_always()
    return r['result']

current_result = ""
def ping_untill_other_response():
    result = send_request_and_get_response()
    current_result = result[0:6]
    
    for x in range(0, 60):
        result2 = send_request_and_get_response()
        compare_result = result2[0:6]
        
        if compare_result != current_result:
            return True
        time.sleep(20) # wait 20 seconds before trying again
        
    # If this returns false, something went completely wrong because in 30 minutes nothing changed?
    return False
    

In [107]:
def lets_do_this():
    stop_looping = False
    
    while stop_looping == False:
        ping_untill_other_response()
        ad = get_advanced_new_data()
        guess_based_on_advanced_data(ad)
        if correct:
            stop_looping = True

In [108]:
lets_do_this()

Static ping 0.00017700763
Static ping 0.00017700763
Static ping 0.00017700763
Static ping 0.000177008
Static ping 0.000177008
Static ping 0.000177008
Static ping 0.00017700763
Static ping 0.00017700763
Static ping 0.000177008
Static ping 0.00017700763
Static ping 0.00017700763
Static ping 0.000177008
Static ping 0.00017700763
Static ping 0.00017700763
Static ping 0.00017700763
Static ping 0.00017700763
Static ping 0.00017700763
Static ping 0.00017700763
Static ping 2.9880894e-05
Getting results...
Got all single results. Total of 974 records.
sorting list of results with 974 elements
{'status': 'ok', 'result': '9.996833e-06', 'label': 'Persian_cat', 'index': 283}
{'status': 'ok', 'result': '9.993992e-06', 'label': 'tiger_shark', 'index': 3}
{'status': 'ok', 'result': '9.96658e-05', 'label': 'revolver', 'index': 763}
{'status': 'ok', 'result': '9.965671e-06', 'label': 'golfcart', 'index': 575}
{'status': 'ok', 'result': '9.944361e-06', 'label': 'projector', 'index': 745}
{'status': 'ok'

:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
label_name: earthstar
label_index: 995
max_value: 9.79
min_value: 3.86
average_total: 7.862499999999999
average_top_three: 9.196666666666665
total_dirs: 4
--------------------
sending guess for label earthstar wit

:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
label_name: thimble
label_index: 855
max_value: 9.58
min_value: 8.24
average_total: 9.120000000000001
average_top_three: 9.413333333333334
total_dirs: 4
--------------------
sending guess for label thimble with synsnet n04423845
Guess was wrong. Status code: 500. With message: b'Error: could not handle the request\n'
answer correct: False
Static ping 0.00022050082
Static ping 0.00022049995
Static ping 0.00022049995
Static ping 0.00022049995
Static ping 0.00022049995
Static ping 0.00022049995
Static ping 0.00022049995
Static ping 0.00022050082
Static ping 0.00022049995
Static ping 0.00022049995
Static ping 0.00022049995
Static ping 0.00022049995
Static 

:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
label_name: flute
label_index: 558
max_value: 9.21
min_value: 5.29
average_total: 8.102500000000001
average_top_three: 9.040000000000001
total_dirs: 4
--------------------
sending guess for label flute with synsnet n03372029
Guess was wrong. Status code: 50

Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
:-( [Errno 2] No such file or directory: ''
:-( [Errno 2] No such file or directory: ''
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
Something went wrong could not convert string to float: 'Coul'
label_name: recreational_vehicle
label_index: 757
max_value: 9.95
min_value: 3.2
average_total: 7.835
average_top_three: 9.38
total_dirs: 4
--------------------
sending guess for label recreational_vehicle with synsnet n04065272
Guess was wrong. Status code: 500. With message: b'Error: could not handle the request\n'
answer correct: False
Static ping 6.313533e-05
Static ping 6.313514e-05
Static ping 6.313514e-05
Static ping 6.313514e-05


KeyboardInterrupt: 

In [114]:
for x in range(0, 1000):
    try:
        y = [x]
        print("doing " + str(x))
        get_adnvanced_data_by_index_list(y)
    except Exception as e:
        print(str(x) + " goes wrong " + str(e)) 

doing 0
doing 1
Something went wrong could not convert string to float: 'Coul'
doing 2
doing 3
doing 4
doing 5
doing 6
doing 7
doing 8
doing 9
doing 10
doing 11
doing 12
doing 13
doing 14
doing 15


KeyboardInterrupt: 

In [115]:
y = [1]
get_adnvanced_data_by_index_list(y)

KeyboardInterrupt: 

In [118]:
print(float("9.0411275e-05"))

9.0411275e-05
